In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('../Train_data.csv')

In [5]:
from sklearn.preprocessing import LabelEncoder
enc_dict={}
def encoding_label(x):
    le = LabelEncoder()
    r = le.fit_transform(x)
    enc_dict[x.name] = le.classes_
    return r

In [6]:
X_cat = df[['protocol_type', 'service', 'flag']]
# X_cat = df.select_dtypes(include=['object'])
X_cat = pd.get_dummies(X_cat)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_num = df.drop(['protocol_type', 'service', 'flag', 'class'], axis=1)
scaler.fit(X_num)
X_scaled = scaler.transform(X_num)
X_scaled = pd.DataFrame(X_scaled, index=X_num.index, columns=X_num.columns)

X = pd.concat([X_scaled, X_cat], axis=1)

y = df[['class']].apply(encoding_label)

In [8]:
from sklearn.model_selection import train_test_split
# train test set 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    stratify=y, 
                                                    random_state=1)
# train, validation분리
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.25,
                                                  stratify=y_train,
                                                  random_state=1)

In [9]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [10]:
def gridSearch(model, param, num):
    start_time=time.time()
    grid_search = GridSearchCV(model,
                               param_grid=param,
                               #scoring='recall',
                               scoring=['accuracy','recall'], 
                               refit='recall',
                               cv=num,
                               #verbose=1,
                               n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print("최고점수 :", grid_search.best_score_)
    print('걸린시간 :', time.time() - start_time)
    return grid_search.best_estimator_, grid_search.best_params_

In [11]:
def randGridSearch(model,param, num):
    start_time=time.time()
    randomized_search = RandomizedSearchCV(model,
                                           param_distributions=param,
                                           n_iter=200,
#                                           scoring='recall',
                                           scoring=['accuracy','recall'], 
                                           refit='recall',
                                           cv=num,
                                           n_jobs=-1)
    randomized_search.fit(X_train,y_train)
    print("최고점수: ",randomized_search.best_score_)
    print('걸린시간 :', time.time() - start_time)

    return randomized_search.best_estimator_, randomized_search.best_params_

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, plot_roc_curve, plot_precision_recall_curve
from sklearn.model_selection import cross_val_score
import time

In [13]:
def evaluating_model(name,model,params, Xdata,ydata):
    pred = model.predict(Xdata)
    cross_score = cross_val_score(model, Xdata, ydata, scoring='recall', cv=5)
    accuracy = accuracy_score(ydata, pred)
    c_matrix = confusion_matrix(ydata, pred)
    classification = classification_report(ydata, pred, target_names=[enc_dict['class'][0],enc_dict['class'][1]])
    print('=============', name, 'evaluating=============')
    print('Best params: \n', params)
    print("Mean Cross_val_score(recall): \n", cross_score.mean())
    print("Accuracy_score: \n", accuracy)
    print("Confusion_matrix: \n", c_matrix)
    print("Classification_report: \n", classification)

# --------------여기서부터 추가할 부분입니다.-----------------------

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# 로지스틱 리그레션

In [15]:
# LogisticRegression
lr = LogisticRegression()
lr_param = {
    'penalty' : ['l2', 'l1'],# 규제 유형: 기본값 'l2'
     'C' : [0.01, 0.1, 1, 5, 10] # 규제 강도: C값이 작을수록 규제 강도가 크다
}

lr_best_estimator, lr_best_params = gridSearch(lr, lr_param, 5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


최고점수 : 0.9821546596166556
걸린시간 : 8.457817792892456


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:
lr.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [17]:
lr_best_estimator, lr_best_params

(LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False), {'C': 1, 'penalty': 'l2'})

In [18]:
evaluating_model("LogisticRegression", lr_best_estimator, lr_best_params, X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

============= LogisticRegression evaluating=============
Best params: 
 {'C': 1, 'penalty': 'l2'}
Mean Cross_val_score(recall): 
 0.9821546596166556
Accuracy_score: 
 0.9739590684544813
Confusion_matrix: 
 [[6362  243]
 [ 126 7439]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       0.98      0.96      0.97      6605
      normal       0.97      0.98      0.98      7565

    accuracy                           0.97     14170
   macro avg       0.97      0.97      0.97     14170
weighted avg       0.97      0.97      0.97     14170



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [19]:
evaluating_model("LogisticRegression", lr_best_estimator, lr_best_params, X_val, y_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

============= LogisticRegression evaluating=============
Best params: 
 {'C': 1, 'penalty': 'l2'}
Mean Cross_val_score(recall): 
 0.9785902876001884
Accuracy_score: 
 0.9693056731583404
Confusion_matrix: 
 [[2097  105]
 [  40 2482]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       0.98      0.95      0.97      2202
      normal       0.96      0.98      0.97      2522

    accuracy                           0.97      4724
   macro avg       0.97      0.97      0.97      4724
weighted avg       0.97      0.97      0.97      4724



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iteratio

# SVM

In [20]:
svc = SVC(kernel='linear', C=0.01, random_state=1)
svc_param = {
    # 'C': [0.001, 0.01, 0.1, 1, 10, 100],
    # 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]
}

svc_best_estimator, svc_best_params = gridSearch(svc, svc_param, 5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


최고점수 : 0.9841374752148051
걸린시간 : 20.3105206489563


In [22]:
svc.get_params().keys()

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

In [23]:
svc_best_estimator, svc_best_params
# {'C': 100, 'gamma': 0.001}

(SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
     max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
     verbose=False), {})

In [21]:
evaluating_model("SVM", svc_best_estimator, svc_best_params, X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

============= SVM evaluating=============
Best params: 
 {}
Mean Cross_val_score(recall): 
 0.9841374752148051
Accuracy_score: 
 0.9721242060691602
Confusion_matrix: 
 [[6327  278]
 [ 117 7448]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       0.98      0.96      0.97      6605
      normal       0.96      0.98      0.97      7565

    accuracy                           0.97     14170
   macro avg       0.97      0.97      0.97     14170
weighted avg       0.97      0.97      0.97     14170



In [24]:
evaluating_model("SVM", svc_best_estimator, svc_best_params, X_val, y_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

============= SVM evaluating=============
Best params: 
 {}
Mean Cross_val_score(recall): 
 0.9785902876001886
Accuracy_score: 
 0.9676121930567316
Confusion_matrix: 
 [[2086  116]
 [  37 2485]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       0.98      0.95      0.96      2202
      normal       0.96      0.99      0.97      2522

    accuracy                           0.97      4724
   macro avg       0.97      0.97      0.97      4724
weighted avg       0.97      0.97      0.97      4724



# 랜덤 포레스트

In [25]:
rf = RandomForestClassifier()
rf_param = {
    'n_estimators' : [10, 100],
    'max_depth' : [5, 7, 9],
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

rf_best_estimator, rf_best_params = gridSearch(rf, rf_param, 5)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


최고점수 : 0.9974884335756775
걸린시간 : 88.11090636253357


In [26]:
rf.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [27]:
rf_best_estimator, rf_best_params

(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=7, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=8, min_samples_split=16,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=None,
                        verbose=0, warm_start=False),
 {'max_depth': 7,
  'min_samples_leaf': 8,
  'min_samples_split': 16,
  'n_estimators': 100})

In [28]:
evaluating_model("RandomForestClassifier", svc_best_estimator, svc_best_params, X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

============= RandomForestClassifier evaluating=============
Best params: 
 {}
Mean Cross_val_score(recall): 
 0.9841374752148051
Accuracy_score: 
 0.9721242060691602
Confusion_matrix: 
 [[6327  278]
 [ 117 7448]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       0.98      0.96      0.97      6605
      normal       0.96      0.98      0.97      7565

    accuracy                           0.97     14170
   macro avg       0.97      0.97      0.97     14170
weighted avg       0.97      0.97      0.97     14170



In [29]:
evaluating_model("RandomForestClassifier", svc_best_estimator, svc_best_params, X_val, y_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

============= RandomForestClassifier evaluating=============
Best params: 
 {}
Mean Cross_val_score(recall): 
 0.9785902876001886
Accuracy_score: 
 0.9676121930567316
Confusion_matrix: 
 [[2086  116]
 [  37 2485]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       0.98      0.95      0.96      2202
      normal       0.96      0.99      0.97      2522

    accuracy                           0.97      4724
   macro avg       0.97      0.97      0.97      4724
weighted avg       0.97      0.97      0.97      4724



# Testing set evaluation

In [30]:
models = []
# models.append(('Decision Tree Classifier', Decision_best_estimator))
# models.append(('KNeighborsClassifier', knn_best_estimator))
# models.append(('GradientBoostingClassifier', gbc_best_estimator)) 
# models.append(('XGBClassifier', xgb_best_estimator))
models.append(('LogisticRegression', lr_best_estimator))
models.append(('SVM', svc_best_estimator))
models.append(('RandomForestClassifier', rf_best_estimator))

In [31]:
for i, v in models:
    pred = v.predict(X_test)
    cross_score = cross_val_score(v, X_test, y_test, scoring='recall', cv=5)
    accuracy = accuracy_score(y_test, pred)
    c_matrix = confusion_matrix(y_test, pred)
    classification = classification_report(y_test, pred, target_names=[enc_dict['class'][0],enc_dict['class'][1]])
    print('=================================', i, ' evaluating=================================')
    print("Mean Cross_val_score(recall): \n", cross_score.mean())
    print("Accuracy_score: \n", accuracy)
    print("Confusion_matrix: \n", c_matrix)
    print("Classification_report: \n", classification)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

================================= LogisticRegression  evaluating=================================
Mean Cross_val_score(recall): 
 0.9821530283733108
Accuracy_score: 
 0.9707843759923785
Confusion_matrix: 
 [[2800  136]
 [  48 3314]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       0.98      0.95      0.97      2936
      normal       0.96      0.99      0.97      3362

    accuracy                           0.97      6298
   macro avg       0.97      0.97      0.97      6298
weighted avg       0.97      0.97      0.97      6298



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

================================= SVM  evaluating=================================
Mean Cross_val_score(recall): 
 0.9857209191254512
Accuracy_score: 
 0.9693553509050492
Confusion_matrix: 
 [[2785  151]
 [  42 3320]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       0.99      0.95      0.97      2936
      normal       0.96      0.99      0.97      3362

    accuracy                           0.97      6298
   macro avg       0.97      0.97      0.97      6298
weighted avg       0.97      0.97      0.97      6298



/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

================================= RandomForestClassifier  evaluating=================================
Mean Cross_val_score(recall): 
 0.9958342177881555
Accuracy_score: 
 0.9842807240393776
Confusion_matrix: 
 [[2841   95]
 [   4 3358]]
Classification_report: 
               precision    recall  f1-score   support

     anomaly       1.00      0.97      0.98      2936
      normal       0.97      1.00      0.99      3362

    accuracy                           0.98      6298
   macro avg       0.99      0.98      0.98      6298
weighted avg       0.98      0.98      0.98      6298

